In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from scipy.io import loadmat, savemat

from tensorflow.keras import layers

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [3]:
Y_4802 = loadmat('Y_4802.mat')['Y_4802']

In [4]:
sequence = loadmat('dataset_4802.mat')['Sequence']

In [5]:
amino_code = {'A':0, 'C':1, 'D':2, 'E':3, 'F':4, 'G':5, 'H':6,
             'I':7, 'K':8, 'L':9, 'M':10, 'N':11, 'P':12, 
             'Q':13, 'R':14, 'S':15, 'T':16, 'U':17, 'V':18,
             'W':19, 'X':20, 'Y':21 }

In [6]:
extend_one_encode = []
seq_num_encode = []
for s in sequence:
    p_seq = (list(s[0][0]) * 1000)[0:10000]
    seq_num = np.array([amino_code[x] for x in p_seq])
    seq_num_encode.append(seq_num)
    tmp = np.zeros((seq_num.size, 22))
    tmp[np.arange(seq_num.size), seq_num] = 1
    extend_one_encode.append(tmp)

In [7]:
X_4802_padding = np.array(extend_one_encode)

In [8]:
X_4802_padding.shape

(4802, 10000, 22)

In [9]:
X_4802_num = np.array(seq_num_encode); X_4802_num.shape

(4802, 10000)

In [10]:
X_4802_padding[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
Y_4802.shape

(4802, 37)

In [12]:
#inputs = keras.Input(shape=(None, 22,), dtype = "float32")
inputs = keras.Input(shape=(None,), dtype = "int32")
x = layers.Embedding(22, 22)(inputs)
x = layers.Bidirectional(layers.LSTM(256, return_sequences = True))(x)
#x = layers.Bidirectional(layers.LSTM(128, return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(128))(x)
x = layers.Reshape((16,16,1))(x)

#x = layers.Conv2D(128, 3, activation='relu',)(x)
x = layers.Conv2D(128, 3, activation='relu',)(x)
x = layers.Conv2D(64, 3, activation='relu',)(x)
x = layers.Conv2D(32, 3, activation='relu',)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(37, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 22)          484       
_________________________________________________________________
bidirectional (Bidirectional (None, None, 512)         571392    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               656384    
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 64)       

In [13]:
#tensorboard_callback = keras.callbacks.TensorBoard()

model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_4802_num, Y_4802, batch_size=32, epochs=2,)
         # callbacks = [tensorboard_callback])#, validation_data=(x_val, y_val))

Epoch 1/2
151/151 [==============================] - 426s 3s/step - loss: 0.1406 - accuracy: 0.1945
Epoch 2/2
151/151 [==============================] - 425s 3s/step - loss: 0.1063 - accuracy: 0.2003


In [14]:
X_4802_feature = loadmat('feature_4802.mat')

In [15]:
pssmpse = X_4802_feature['pssmpse']

In [16]:
pssmpse[0].shape

(31, 20, 2)

In [17]:
X_4802_feature.keys()

dict_keys(['__header__', '__version__', '__globals__', 'ppab', 'ppdwt', 'pppse', 'pssmab', 'pssmdwt', 'pssmpse', 'label'])

In [18]:
pppse = X_4802_feature['pppse']

In [19]:
pppse.shape

(4802, 31, 57, 2)

In [20]:
input_shape = (4, 7, 28, 28, 3)
input_shape[2:]

(28, 28, 3)

In [21]:
input_shape = pssmpse.shape
pssm = keras.Input(shape=input_shape[1:], dtype = "float32")

x = layers.Conv2D(256, (4,3), activation='relu')(pssm)
x = layers.Conv2D(128, 3, activation='relu',)(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',)(x)
#x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3, activation='relu',)(x)
outputs = layers.Flatten()(x)
#outputs = layers.Dense(37, activation='sigmoid')(x)
model_pssm = keras.Model(pssm, outputs)
model_pssm.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 31, 20, 2)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 18, 256)       6400      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 16, 128)       295040    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 8, 128)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 6, 64)         73792     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 9, 4, 32)          18464     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)             

In [22]:
input_shape = pppse.shape
pse = keras.Input(shape=input_shape[1:], dtype = "float32")

x = layers.Conv2D(256, (4,3), activation='relu')(pse)
x = layers.Conv2D(128, 3, activation='relu',)(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu',)(x)
#x = layers.MaxPooling2D()(x)
x = layers.Conv2D(32, 3, activation='relu',)(x)
outputs = layers.Flatten()(x)
#outputs = layers.Dense(37, activation='sigmoid')(x)
model_pse = keras.Model(pse, outputs)
model_pse.summary()

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 31, 57, 2)]       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 28, 55, 256)       6400      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 26, 53, 128)       295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 26, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 24, 64)        73792     
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 9, 22, 32)         18464     
_________________________________________________________________
flatten_2 (Flatten)          (None, 6336)             

In [23]:
tmodel = layers.Concatenate()([model_pssm.output, model_pse.output])
x = layers.Dense(37, activation='sigmoid')(tmodel)
model = keras.Model(inputs=[pssm, pse], outputs = x)
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 31, 20, 2)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 31, 57, 2)]  0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 28, 18, 256)  6400        input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 28, 55, 256)  6400        input_3[0][0]                    
_______________________________________________________________________________________

In [24]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit([pssmpse, pppse], Y_4802, batch_size=8, epochs=50,validation_split=0.1)
         # callbacks = [tensorboard_callback])#, validation_data=(x_val, y_val))

Epoch 1/50
541/541 [==============================] - 9s 17ms/step - loss: 0.1139 - accuracy: 0.1937 - val_loss: 0.0879 - val_accuracy: 0.2682
Epoch 2/50
541/541 [==============================] - 8s 15ms/step - loss: 0.1086 - accuracy: 0.1928 - val_loss: 0.0904 - val_accuracy: 0.2682
Epoch 3/50
541/541 [==============================] - 8s 15ms/step - loss: 0.1080 - accuracy: 0.1928 - val_loss: 0.0910 - val_accuracy: 0.2682
Epoch 4/50
541/541 [==============================] - 8s 15ms/step - loss: 0.1071 - accuracy: 0.1976 - val_loss: 0.0904 - val_accuracy: 0.2682
Epoch 5/50
541/541 [==============================] - 8s 15ms/step - loss: 0.1048 - accuracy: 0.2171 - val_loss: 0.0899 - val_accuracy: 0.3222
Epoch 6/50
541/541 [==============================] - 8s 15ms/step - loss: 0.1030 - accuracy: 0.2344 - val_loss: 0.0847 - val_accuracy: 0.3576
Epoch 7/50
541/541 [==============================] - 8s 15ms/step - loss: 0.0999 - accuracy: 0.2534 - val_loss: 0.0787 - val_accuracy: 0.4262

In [25]:
res = model.predict([pssmpse[0:10], pppse[0:10]])

In [26]:
max(res.flatten())

0.9999515

In [27]:
min(res.flatten())

2.9882393e-38

In [28]:
res[res>=0.5] = 1
res[res<0.5] = 0

In [29]:
res

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,